<a href="https://colab.research.google.com/github/kipsangmarion/webscraper/blob/main/WebScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dependencies

In [ ]:
from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd
import numpy as np
import csv

#Extracting the links to individual project details

In [ ]:
# Replace with the URL of the website you want to scrape
urls = ['https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=0',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=1',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=2',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=3',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=4',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=5',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=6',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=7',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=8',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=9',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=10',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=11',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=12',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=13',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=14',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=15',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=16',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=17',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=18',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=19',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=20',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=21',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=22',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=23',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=24',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=25',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=26',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=27',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=28',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=29',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=30',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=31',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=32',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=33',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=34',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=35',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=36',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=37',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=38',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=39',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=40',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=41',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=42',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=43',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=44',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=54',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=46',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=47',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=48',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=49',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=50',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=51',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=52',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=53',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=54',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=55',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=56',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=57',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=58',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=59',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=60',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=61',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=62',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=63',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=64',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=65',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=66',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=67',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=68',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=69',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=70',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=71',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=72',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=73',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=74',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=75',
        'https://www.thegef.org/projects-operations/database?f%5B0%5D=focal_areas%3A2207&search=&page=76'
        ]

# Send an HTTP request to the URL
project_links = []
for url in urls:
  response = requests.get(url)

  soup = BeautifulSoup(response.text, 'html.parser')

  # Find the table that contains the links (you may need to inspect the HTML)
  table = soup.find('table')

  # Loop through rows in the table
  for row in table.find_all('tr'):
    # Check if there are any <td> elements in this row
    td_elements = row.find_all('td')
    if td_elements:
      # Assuming the links are in the first column (index 0)
      link = td_elements[0].find('a')
      if link:
        link_text = link.get('href')
        full_link = "https://www.thegef.org" + link_text
        project_links.append(full_link)

# If your computer takes too long to compute you have faster access to the data
link_dataframe = pd.DataFrame(project_links)
link_dataframe.to_csv('project_links.csv', index= False, header= None)

project_links

#Extract project general info

Including catching any exceptions

In [ ]:
from requests.sessions import ChunkedEncodingError
project_title = []
funding_institution = []
implementing_institution = []
country = []
region = []
status = []
total_project_amount = []
link_to_project_documentation = []

retry_count = 3
for attempt in range(retry_count):
  try:
    for project_link in project_links:
      response = requests.get(project_link)

      soup = BeautifulSoup(response.text, 'html.parser')

      # Project title
      try:
        title_element = soup.find('span', class_='field--name-title')
        title_text = title_element.get_text(strip=True)
      except:
        title_text = None
      project_title.append(title_text)

      # Funding institution
      try:
        funding_institution_element = soup.find('div', class_='field--name-field-trust-fund-name')
        funding_institution_item = funding_institution_element.find_all('div', class_='field__item')
        funding_institution_list = [item.get_text(strip=True) for item in funding_institution_item]
      except:
        funding_institution_list = None
      funding_institution.append(funding_institution_list)

      # Implementing institution
      try:
        implementing_institution_element = soup.find('div', class_='field--name-field-implementing-agencies')
        implementing_institution_item = implementing_institution_element.find_all('div', class_='field__item')
        implementing_institution_list = [item.get_text(strip=True) for item in implementing_institution_item]
      except:
        emplementing_institution_list = None
      implementing_institution.append(implementing_institution_list)

      # Country
      try:
        country_element = soup.find('div', class_='field--name-field-country')
        country_item = country_element.find_all('div', class_='field__item')
        country_list = [item.get_text(strip=True) for item in country_item]
      except:
        country_list = None
      country.append(country_list)

      # Region
      try:
        region_element = soup.find('div', class_='field--name-field-region')
        region_item = region_element.find_all('div', class_='field__item')
        region_list = [item.get_text(strip=True) for item in region_item]
      except:
        region_list = None
      region.append(region_list)

      # Status
      try:
        status_element = soup.find('div', class_='field--name-field-status')
        status_text = status_element.find('div', class_='field__item').get_text(strip = True)
      except:
        status_text = None
      status.append(status_text)

      # Total amount
      try:
        gef_project_grant_element = soup.find('div', class_='views-field-field-gef-project-grant')
        cofinancing_element = soup.find('div', class_ ='views-field-field-co-financing-total')

        gef_project_grant_amount = gef_project_grant_element.find('div', class_='field-content').get_text(strip=True)
        cofinancing_amount = cofinancing_element.find('div', class_ ='field-content').get_text(strip=True)
        total_amount = int(gef_project_grant_amount) + int(cofinancing_amount)
      except:
        total_amount = None

      total_project_amount.append(total_amount)

      # Link to project documentation
      try:
        project_documents_element = soup.find('div', class_='field--name-field-document-url')
        project_documents_items = project_documents_element.find_all('div', class_='field__item')
        document_links = [item.find('a')['href'] for item in project_documents_items]
      except:
        document_links = None
      link_to_project_documentation.append(document_links)

  except ChunkedEncodingError as e:
    print(f"ChunkedEncodingError: {e}")
    if attempt < retry_count - 1:
      print("Retrying...")
    else:
      print("Max retry attempts reached.")
  except requests.RequestException as e:
    print(f"RequestException: {e}")
    break



ChunkedEncodingError: ('Connection broken: IncompleteRead(8458 bytes read, 1782 more expected)', IncompleteRead(8458 bytes read, 1782 more expected))
Retrying...
ChunkedEncodingError: ('Connection broken: IncompleteRead(784 bytes read, 4652 more expected)', IncompleteRead(784 bytes read, 4652 more expected))
Retrying...


Hey

#Create dataframe and export to csv

For the purposes of downloading

In [ ]:
ss = pd.DataFrame(
    {'project_title': project_title, 'funding_institution': funding_institution, 'implementing_institution': implementing_institution,
     'country': country, 'region': region,'status': status, 'total_project_amount': total_project_amount,
     'link_to_project_documentation': link_to_project_documentation})
ss.head()
ss.to_csv('thegef_data.csv')